# 1. **패키지 불러오기**

In [1]:
###########################################################################################
## 필요한 모듈 패키지 불러오기
###########################################################################################

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
from sklearn.model_selection import train_test_split
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time, math, datetime, requests, json

pd.options.display.float_format = '{:,.2f}'.format # 소수점 2자리에서 반올림, 천단위 쉼표

# 2. **데이터 소스 수집**

## 1) 데이터소스 1: 발전실적

In [2]:
###########################################################################################
## 크롤링: CLASS 정의: 발전실적 가져오는 클래스 output 정의
###########################################################################################

class output:

    def get_output(self, site, year, month, day):
    
        ## 키보드 설정

        self.ENTER = '/ue007'
        self.BACKSPACE = '/ue003'
        self.LEFT = '/ue012'
        self.UP = '/ue013'
        self.RIGHT ='/ue014'
        self.DOWN = '/ue015'

        ## 크롬을 이용해서 사이트 불러오기

        self.url = 'https://www.solarcube.co.kr/SrMain/SM010.aspx'
        self.driver = webdriver.Chrome('e:/코딩/python/chromedriver')
        self.driver.get(self.url)

        ## 사이트에서 원하는 정보 가져오기

        self.driver.find_element_by_xpath('//*[@id="Txt_1"]').send_keys('sclossolar')
        time.sleep(1.0)
        self.driver.find_element_by_xpath('//*[@id="Txt_2"]').send_keys('sclossolar')
        time.sleep(1.0)
        self.driver.find_element_by_xpath('//*[@id="imageField"]').click()
        time.sleep(1.0)
        self.driver.find_element_by_xpath('//*[@id="SrTop_Panel1"]/div[2]/ul/li[5]/a').click()
        time.sleep(1.0)
        self.driver.find_element_by_xpath('//*[@id="select2-SrTop_cbo_plant_d1-container"]').click()
        time.sleep(1.0)
        self.web1 = self.driver.find_element_by_xpath('/html/body/span/span/span[1]/input')
        self.web1.send_keys('%s' % site)
        time.sleep(1.0)
        self.web1.send_keys(Keys.ENTER)
        time.sleep(1.0)
        self.web2 = self.driver.find_element_by_xpath('//*[@id="txt_Calendar"]')
        self.web2.click()
        time.sleep(1.0)
        for i in range(0,10):
            self.web2.send_keys(Keys.RIGHT)
        for i in range(0,10):
            self.web2.send_keys(Keys.BACKSPACE)
        self.web2.send_keys('%s-%s-%s' % (year, month, day))
        time.sleep(0.5)
        self.driver.find_element_by_xpath('//*[@id="submitbtn"]').click()
        time.sleep(1.0)

        ## 전체 내용 불러오기

        self.html = BeautifulSoup(self.driver.page_source, 'html.parser') # html 전체 내용 불러오기
        self.data = self.html.select('div.sub_contents > div.s_cont > table.sub_com_table > tbody > tr > td') # 발전실적 테이블 추출

        ## 크롬창 닫기

        self.driver.close()

        # 발전량 데이터 전처리 및 데이터프레임 만들기

        self.data_text = []

        for i in range(len(self.data)):
            self.a = str(self.data[i]).replace('<td>', '')
            self.b = self.a.replace('</td>', '')
            self.c = self.b.replace('<span class="f_w600">', '')
            self.d = self.c.replace('</span>', '')
            self.e = self.d.replace('<span class="align_C">', '')
            self.f = self.e.replace('<span class="align_R">', '')
            self.data_text.append(self.f)

        self.data_num = len(self.data_text) // 4

        self.data_array = np.array(self.data_text) # 리스트를 넘파이 어레이화
        self.data_array = self.data_array.reshape(int(self.data_num), 4) # nx4 행렬로 변환
        self.output_yesterday = pd.DataFrame(self.data_array, columns = ['site', 'date', 'output', 'radiation']) # 행렬을 데이터프레임으로 변환


        return self.output_yesterday

## 2) 데이터 소스 2: 기상데이터

In [3]:
###########################################################################################
## 크롤링: CLASS 정의:기상예보 클래스 forecast 정의
###########################################################################################

class forecast:
    
    # 기상 데이터 불러오기 함수 get_forecast() 정의

    def get_forecast(self, base_date, rows, base_time, nx, ny):
        self.servicekey = 'C5XWwKpv9QPJ8%2FX1nDI7qY2ZgVrDp9r4fzIXYCvjfj5uZ6MbPHY3SxTJkzrOpzDUrYW5j4%2FEJTklpG3JZzhigg%3D%3D'
        self.str = 'http://apis.data.go.kr/1360000/VilageFcstInfoService_2.0/getVilageFcst?ServiceKey={servicekey}&pageNo={page}&numOfRows={rows}&dataType={data_type}&base_date={base_date}&base_time={base_time}&nx={nx}&ny={ny}'
        self.url = self.str.format(servicekey=self.servicekey, page='1', rows=rows, data_type='json', base_date=base_date, base_time=base_time, nx=nx, ny=ny)
        self.resp = requests.get(self.url)
        self.data = json.loads(self.resp.text)
        return self.data

    # 불러온 데이터 정리하기 함수 make_df() 정의

    def make_forecast_df(self, data):
        self.data_items = data['response']['body']['items']['item']
        
        self.df_data = pd.DataFrame(self.data_items)
        self.df_data_sort = self.df_data.sort_values(by = 'category')

        # 예보항목 = ['TMP(온도)', 'WSD(풍속)', 'REH(습도)', 'SKY(운량)']

        self.TMP = self.df_data_sort.loc[self.df_data_sort.category == 'TMP', :]
        self.TMP= self.TMP.sort_index()
        self.TMP.columns = ['announce_date', 'announce_time', 'announce', 'forecast_date', 'forecast_time', 'temperature', 'x', 'y']
        #self.TMP = self.TMP.set_index('forecast_time')
        self.TMP = self.TMP.drop(['announce', 'x', 'y'], axis = 1)

        self.PCP = self.df_data_sort.loc[self.df_data_sort.category == 'PCP', :]
        self.PCP = self.PCP.sort_index()
        self.PCP = self.PCP.drop(['baseDate', 'baseTime', 'category', 'nx', 'ny'], axis = 1)
        self.PCP.columns = ['forecast_date', 'forecast_time', 'rainfall']
        self.PCP = self.PCP.set_index('forecast_time')
        self.PCP['rainfall'].replace('강수없음', '0', inplace = True)
        self.PCP['rainfall'].replace('1mm 미만', '0', inplace = True)
        self.PCP['rainfall'].replace('30~50mm', '40', inplace = True)
        self.PCP['rainfall'].replace('50mm 이상', '50', inplace = True)
        self.PCP['rainfall'] = self.PCP['rainfall'].str.replace('mm', '')
        self.PCP['rainfall'].astype('float64')
        self.forecast = pd.merge(self.TMP, self.PCP, how = 'left', on = ['forecast_time', 'forecast_date'])

        self.WSD = self.df_data_sort.loc[self.df_data_sort.category == 'WSD', :]
        self.WSD = self.WSD.sort_index()
        self.WSD = self.WSD.drop(['baseDate', 'baseTime', 'category', 'nx', 'ny'], axis = 1)
        self.WSD.columns = ['forecast_date', 'forecast_time', 'windvelocity']
        #self.WSD = self.WSD.set_index('forecast_time')
        self.forecast = pd.merge(self.forecast, self.WSD, how = 'left', on = ['forecast_time', 'forecast_date'])

        self.REH = self.df_data_sort.loc[self.df_data_sort.category == 'REH', :]
        self.REH = self.REH.sort_index()
        self.REH = self.REH.drop(['baseDate', 'baseTime', 'category', 'nx', 'ny'], axis = 1)
        self.REH.columns = ['forecast_date', 'forecast_time', 'humidity']
        #self.REH = self.REH.set_index('forecast_time')
        self.forecast = pd.merge(self.forecast, self.REH, how = 'left', on = ['forecast_time', 'forecast_date'])

        self.SNO = self.df_data_sort.loc[self.df_data_sort.category == 'SNO', :]
        self.SNO = self.SNO.sort_index()
        self.SNO = self.SNO.drop(['baseDate', 'baseTime', 'category', 'nx', 'ny'], axis = 1)
        self.SNO.columns = ['forecast_date', 'forecast_time', 'snowfall']
        self.SNO = self.SNO.set_index('forecast_time')
        self.SNO['snowfall'].replace('적설없음', '0', inplace = True)
        self.SNO['snowfall'].replace('1cm 미만', '0', inplace = True)
        self.SNO['snowfall'].replace('1cm미만', '0', inplace = True)
        self.SNO['snowfall'].replace('5cm 이상', '5', inplace = True)
        self.SNO['snowfall'].replace('5cm이상', '5', inplace = True)
        self.SNO['snowfall'] = self.SNO['snowfall'].str.replace('mm', '')
        self.SNO['snowfall'].astype('float64')
        self.forecast = pd.merge(self.forecast, self.SNO, how = 'left', on = ['forecast_time', 'forecast_date'])

        self.SKY = self.df_data_sort.loc[self.df_data_sort.category == 'SKY', :]
        self.SKY = self.SKY.sort_index()
        self.SKY = self.SKY.drop(['baseDate', 'baseTime', 'category', 'nx', 'ny'], axis = 1)
        self.SKY.columns = ['forecast_date', 'forecast_time', 'sky']
        #self.SKY = self.SKY.set_index('forecast_time')
        self.forecast = pd.merge(self.forecast, self.SKY, how = 'left', on = ['forecast_time', 'forecast_date'])
    
        return self.forecast

In [4]:
## 기상 데이터 클래스 weather 정의

class weather:
    
    # 기상 데이터 불러오기 함수 get_forecast() 정의

    def get_weather(self, rows, start_date, start_hour, end_date, end_hour, location):
        self.servicekey = 'C5XWwKpv9QPJ8%2FX1nDI7qY2ZgVrDp9r4fzIXYCvjfj5uZ6MbPHY3SxTJkzrOpzDUrYW5j4%2FEJTklpG3JZzhigg%3D%3D'
        self.str = 'http://apis.data.go.kr/1360000/AsosHourlyInfoService/getWthrDataList?serviceKey={servicekey}&numOfRows={rows}&pageNo={page}&dataType={data_type}&dataCd=ASOS&dateCd=HR&stnIds={location}&endDt={end_date}&endHh={end_hour}&startHh={start_hour}&startDt={start_date}'
        self.url = self.str.format(servicekey=self.servicekey, page='1', rows=rows, data_type='json', start_date=start_date, start_hour=start_hour, end_date=end_date, end_hour=end_hour, location=location)
        self.resp = requests.get(self.url)
        self.data = json.loads(self.resp.text)
        return self.data

    # 불러온 데이터 정리하기 함수 make_weather_df() 정의
    
    def make_weather_df(self, data):
        self.data_items = data['response']['body']['items']['item']
        
        self.df_data = pd.DataFrame(self.data_items)
        self.weather_df = self.df_data.drop(['stnId', 'rnum', 'taQcflg', 'rnQcflg', 'wsQcflg', 'wd', 'wdQcflg', 'hmQcflg', 'pv', 'td', 'pa', 'paQcflg', 'ps', 'psQcflg', 'ss', 'ssQcflg', 'icsr', 'hr3Fhsc', 'dc10LmcsCa', 'clfmAbbrCd', 'lcsCh', 'vs', 'gndSttCd', 'dmstMtphNo', 'ts', 'tsQcflg', 'm005Te', 'm01Te', 'm02Te', 'm03Te'], axis = 1)
        self.weather_df.columns = ['date', 'location', 'temperature', 'rainfall', 'windvelocity', 'humidity', 'snowfall', 'sky']
        return self.weather_df

### 날짜 함수 정의

In [5]:
###########################################################################################
## FUNCTION 정의:날짜 구하는 함수 thedaybefore(), yesterday(), today(), tomorrow() 정의
###########################################################################################
    
def thedaybefore():
    cdate = datetime.datetime.today()
    pdate = cdate - datetime.timedelta(days = 2)
    p_year = str(pdate.strftime("%y"))
    p_month = str(pdate.strftime("%m"))
    p_day = str(pdate.strftime("%d"))
    thedaybefore = '20' + p_year + p_month + p_day
        
    return thedaybefore

def yesterday():
    cdate = datetime.datetime.today()
    pdate = cdate - datetime.timedelta(days = 1)
    p_year = str(pdate.strftime("%y"))
    p_month = str(pdate.strftime("%m"))
    p_day = str(pdate.strftime("%d"))
    yesterday = '20' + p_year + p_month + p_day
        
    return yesterday

def today():

    cdate = datetime.datetime.today()
    year = str(cdate.strftime("%y"))
    month = str(cdate.strftime("%m"))
    day = str(cdate.strftime("%d"))
    today = '20' + year + month + day
    
    return today

def tomorrow():
    
    cdate = datetime.datetime.today()
    ndate = cdate + datetime.timedelta(days = 1)
    n_year = str(ndate.strftime("%y"))
    n_month = str(ndate.strftime("%m"))
    n_day = str(ndate.strftime("%d"))
    tomorrow = '20' + n_year + n_month + n_day
    
    return tomorrow

## 3) 데이터 소스 3: 태양 위치 관련 column

In [6]:
###########################################################################################
## FUNCTION 정의:태양위치 구하는 함수 solar() 정의
###########################################################################################

def solar(local_latitude, local_longitude, standard_longitude, start_day):
    
    # 각도 단위 변경

    d2r = math.pi / 180
    r2d = 180 / math.pi
    
    date = []
    hour = []
    altitude = []
    altitude_before = []
    azimuth = []
    azimuth_2 = []
    azimuth_2_before = []
    azimuth_final = []

    for i in range(0, int(n_hours)):
    
    
        # 계산 시각
        
        cal_day = start_day + datetime.timedelta(hours = i)
        year = int(cal_day.strftime("%y"))
        month = int(cal_day.strftime("%m"))
        day = int(cal_day.strftime("%d"))
        local_hour = int(cal_day.strftime("%H"))
        local_min = int(cal_day.strftime("%M"))
    
        date.append(int('20' + cal_day.strftime("%y%m%d%H%M")))
        hour.append(int(local_hour))        
        
        # 계산 시각 -1 시간

        cal_day_before = start_day + datetime.timedelta(hours = i - 1)
        year_before = int(cal_day_before.strftime("%y"))
        month_before = int(cal_day_before.strftime("%m"))
        day_before = int(cal_day_before.strftime("%d"))
        local_hour_before = int(cal_day_before.strftime("%H"))
        local_min_before = int(cal_day_before.strftime("%M"))

        # 균시차

        day_of_year = datetime.datetime(year, month, day).timetuple().tm_yday
        B = (day_of_year - 1) * 360 / 365
        EOT = 229.2 * (0.000075
                       + 0.001868 * math.cos(d2r * B)
                       - 0.032077 * math.sin(d2r * B)
                       - 0.014615 * math.cos(d2r * 2 * B)
                       - 0.04089 * math.sin(d2r * 2 * B))

        day_of_year_before = datetime.datetime(year_before, month_before, day_before).timetuple().tm_yday
        B_before = (day_of_year_before - 1) * 360 / 365
        EOT_before = 229.2 * (0.000075
                       + 0.001868 * math.cos(d2r * B_before)
                       - 0.032077 * math.sin(d2r * B_before)
                       - 0.014615 * math.cos(d2r * 2 * B_before)
                       - 0.04089 * math.sin(d2r * 2 * B_before))

        # 시간각

        local_hour_decimal = local_hour + local_min / 60
        delta_longitude = local_longitude - standard_longitude
        solar_time_decimal = (local_hour_decimal * 60 + 4 + delta_longitude + EOT) / 60
        solar_time_hour = int(solar_time_decimal)
        solar_time_min = (solar_time_decimal * 60) % 60
        hour_angle = (local_hour_decimal * 60 + 4 * delta_longitude + EOT) / 60 * 15 - 180

        local_hour_decimal_before = local_hour_before + local_min_before / 60
        delta_longitude = local_longitude - standard_longitude
        solar_time_decimal_before = (local_hour_decimal_before * 60 + 4 + delta_longitude + EOT_before) / 60
        solar_time_hour_before = int(solar_time_decimal_before)
        solar_time_min_before = (solar_time_decimal_before * 60) % 60
        hour_angle_before = (local_hour_decimal_before * 60 + 4 * delta_longitude + EOT_before) / 60 * 15 - 180

        # 적위

        solar_declination = 23.45 * math.sin(d2r * 360 / 365 * (284 + day_of_year))
        
        solar_declination_before = 23.45 * math.sin(d2r * 360 / 365 * (284 + day_of_year_before))

        # 태양 고도

        term_1 = math.cos(d2r * local_latitude) * math.cos(d2r * solar_declination) * math.cos(d2r * hour_angle) + math.sin(d2r * local_latitude) * math.sin(d2r * solar_declination)
        solar_altitude = r2d * math.asin(term_1)
        altitude.append(solar_altitude)
        
        term_1_before = math.cos(d2r * local_latitude) * math.cos(d2r * solar_declination_before) * math.cos(d2r * hour_angle_before) + math.sin(d2r * local_latitude) * math.sin(d2r * solar_declination_before)
        solar_altitude_before = r2d * math.asin(term_1_before)
        altitude_before.append(solar_altitude_before)

        # 태양 방위각 1

        term_2 = (math.sin(d2r * solar_altitude) * math.sin(d2r * local_latitude)- math.sin(d2r * solar_declination)) / (math.cos(d2r * solar_altitude) * math.cos(d2r * local_latitude))
        solar_azimuth = r2d * math.acos(term_2)
        azimuth.append(solar_azimuth)
        
        # Solar Zenith Angle
        
        solar_zenith_angle = r2d * math.acos(math.sin(d2r * local_latitude) * math.sin(d2r * solar_declination) + math.cos(d2r * local_latitude) * math.cos(d2r * solar_declination) * math.cos(d2r * hour_angle))

        solar_zenith_angle_before = r2d * math.acos(math.sin(d2r * local_latitude) * math.sin(d2r * solar_declination_before) + math.cos(d2r * local_latitude) * math.cos(d2r * solar_declination_before) * math.cos(d2r * hour_angle_before))
    
        # 태양 방위각 2
    
        term_3 = (math.sin(d2r * solar_declination) * math.cos(d2r * local_latitude) - math.cos(d2r * hour_angle) * math.cos(d2r * solar_declination) * math.sin(d2r * local_latitude)) / math.sin(d2r * solar_zenith_angle)
        solar_azimuth_2 = r2d * math.acos(term_3)
        azimuth_2.append(solar_azimuth_2)
        
        term_3_before = (math.sin(d2r * solar_declination_before) * math.cos(d2r * local_latitude) - math.cos(d2r * hour_angle_before) * math.cos(d2r * solar_declination_before) * math.sin(d2r * local_latitude)) / math.sin(d2r * solar_zenith_angle_before)
        solar_azimuth_2_before = r2d * math.acos(term_3_before)
        azimuth_2_before.append(solar_azimuth_2_before)
        
        azimuth_delta = solar_azimuth_2 - solar_azimuth_2_before

        if azimuth_delta <= 0:
            solar_azimuth_final = 360 - solar_azimuth_2
        else:
            solar_azimuth_final = solar_azimuth_2
        
        azimuth_final.append(solar_azimuth_final)
    
    solar = pd.DataFrame(date)
    solar['hour'] = hour
    solar['altitude'] = altitude
    solar['altitude_before'] = altitude_before
    solar['altitude_change'] = solar['altitude'] - solar['altitude_before']
    solar['azimuth'] = azimuth_final
    solar = solar.drop(['altitude_before'], axis = 1)
    
    return solar

## 변수정의

In [7]:
###########################################################################################
## VARIABLE 정의: 예측에 필요한 변수 정의
###########################################################################################

# 사이트 리스트 [name, site, capacity, local_latitude, local_longitude, nx, ny, location]
# osan = {'name' : 'osan', 'site' : '오산 삼천리 연구소', 'capacity' : 146.52, 'local_latitude' : 37.18301208, 'local_longitude' : 127.0320559, 'nx' : 61, 'ny' : 118}
# kwangmyoung = {'name' : 'kwangmyoung', 'site' : '삼천리 광명열병합사업단', 'capacity' : 173.4, 'local_latitude' : 37.422222, 'local_longitude' : 126.8910605, 'nx' : 58, 'ny' : 124}

site_list = [['osan', '오산 삼천리 연구소', 146.52, 37.18301208, 127.0320559, 61, 118, 119],
             ['kwangmyoung', '삼천리 광명열병합사업단', 173.4, 37.422222, 126.8910605, 58, 124, 119]]

# 발전실적 가져오는데 필요한 날짜, 시간

yesterday = yesterday()
today = today()
tomorrow = tomorrow()
rows = 500
base_time_1st = "0800"
base_time_2nd = "1400"

# -기상 데이터 가져오는데 필요한 날짜, 시간

start_date = yesterday
start_hour = '00'
end_date = yesterday
end_hour = '23'

# -태양위치 가져오는데 필요한 날짜, 시간

date_today = datetime.datetime.today()
hour_today = int(date_today.strftime('%H'))

date_tomorrow = date_today + datetime.timedelta(days = 1)
year_tomorrow = int('20' + date_tomorrow.strftime('%y'))
month_tomorrow = int(date_tomorrow.strftime('%m'))
day_tomorrow = int(date_tomorrow.strftime('%d'))

# -발전실적 가져오는데 필요한 날짜, 시간

date_yesterday = date_today - datetime.timedelta(days = 1)
year_yesterday = '20' + date_yesterday.strftime('%y')
month_yesterday = date_yesterday.strftime('%m')
day_yesterday = date_yesterday.strftime('%d')


s_year = year_tomorrow
s_month = month_tomorrow
s_day = day_tomorrow
s_local_hour = 0
s_local_min = 0

e_year = year_tomorrow
e_month = month_tomorrow
e_day = day_tomorrow
e_local_hour = 23
e_local_min = 0

start_day = datetime.datetime(s_year, s_month, s_day)
end_day = datetime.datetime(e_year, e_month, e_day)

n = start_day - end_day
n_hours = - n.total_seconds() / 3600 + 24

 # -그래프 표현에 필요한 변수 정의

graph_hours = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23]


# 3. **분석 데이터 셋 구성(site별)**

In [8]:
# -예측 사이트 선택(오산, 광명)

for site_sq in range(0, len(site_list)):
    name = site_list[site_sq][0]
    site = site_list[site_sq][1]
    capacity = site_list[site_sq][2]
    local_latitude = site_list[site_sq][3]
    local_longitude = site_list[site_sq][4]
    nx = site_list[site_sq][5]
    ny = site_list[site_sq][6]
    location = site_list[site_sq][7]
    standard_longitude = 135
 

    ## 어제 발전실적 가져오기

    g_output = output()
    g_output_yesterday = g_output.get_output(site, year_yesterday, month_yesterday, day_yesterday)
    g_output_yesterday['name']=name
    
    #g_output_yesterday.to_csv('./raw/data/%s_raw_elec_(%s%s%s).csv' % (name,year_yesterday,month_yesterday,day_yesterday), encoding='cp949')

    ## 어제 기상데이터 가져오기 및 전운량 데이터(10분위)를 4분위로 변경

    g_weather = weather()
    weather_yesterday = g_weather.get_weather(rows, start_date, start_hour, end_date, end_hour, location)
    weather_yesterday_df = g_weather.make_weather_df(weather_yesterday)
    #weather_yesterday_df['date'] = pd.to_datetime(weather_yesterday_df['date'])
    #weather_yesterday_df['hour'] = weather_yesterday_df['date'].dt.hour
    #weather_yesterday_df['hour'] = weather_yesterday_df['hour'].astype(int)
    #index_hour = pd.DataFrame(graph_hours, columns = ['hour'])
    #weather_yesterday_df = pd.merge(index_hour, weather_yesterday_df, how = 'left', on = ['hour'])
    #weather_yesterday_df['date'] = weather_yesterday_df['date'].fillna('2022-03-01 00:00:00')
    #weather_yesterday_df['date'] = pd.to_datetime(weather_yesterday_df['date'])
    #weather_yesterday_df = weather_yesterday_df.fillna(0)
    weather_yesterday_df['hour'] = graph_hours
    weather_yesterday_df['rainfall'].replace('', 0, inplace = True)
    weather_yesterday_df['sky'].replace('', 0, inplace = True)
    weather_yesterday_df['snowfall'].replace('', 0, inplace = True)
    sky_yesterday = list(weather_yesterday_df.loc[:,'sky'])
    sky_yesterday_modified = []
    for k in sky_yesterday:
        if 0 <= int(k) <= 2:
            sky = 1
        elif 3 <= int(k) <= 5:
            sky = 2
        elif 6 <= int(k) <= 8:
            sky = 3
        else:
            sky = 4
        sky_yesterday_modified.append(sky)
    weather_yesterday_df['sky_modified'] = sky_yesterday_modified
   
    #weather_yesterday_df.to_csv('./raw/data/%s_raw_weather_(%s%s%s).csv' % (name, year_yesterday, month_yesterday, day_yesterday),encoding='cp949')


    ## 태양 위치 계산

    solar_altitude = solar(local_latitude, local_longitude, standard_longitude, start_day)
    solar_altitude.columns = ['date', 'hour', 'altitude', 'altitude_change', 'azimuth']
    
    #solar_altitude.to_csv('./raw/data/%s_raw_solar_(%s%s%s).csv' % (name, year_yesterday, month_yesterday, day_yesterday),encoding='cp949')
    
    ##############################################################################################   
    ## 분석 데이터 셋 병합
    ##############################################################################################
    train_data_df = solar_altitude
    train_data_df2 = pd.merge(train_data_df, weather_yesterday_df, how='right', on=['hour'])
    train_data_df2.rename(columns={'date_y':'date'}, inplace = True)
    train_data_df3 = pd.merge(train_data_df2, g_output_yesterday, how='left', on=['date'])
    
    train_data_df3['site'].fillna(site, inplace = True)
    train_data_df3['output'].fillna(0, inplace = True)
    train_data_df3['radiation'].fillna(0, inplace = True)
    train_data_df3['name'].fillna(name, inplace = True)
    
    train_data_df3.to_csv('./raw/data/%s_raw_train_(%s%s%s).csv' % (name, year_yesterday, month_yesterday, day_yesterday), encoding='cp949')
   

C:\Users\owner\AppData\Local\Temp/ipykernel_16924/1003527085.py:21: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  self.driver = webdriver.Chrome('e:/코딩/python/chromedriver')
C:\Users\owner\AppData\Local\Temp/ipykernel_16924/1003527085.py:26: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  self.driver.find_element_by_xpath('//*[@id="Txt_1"]').send_keys('sclossolar')
C:\Users\owner\AppData\Local\Temp/ipykernel_16924/1003527085.py:28: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  self.driver.find_element_by_xpath('//*[@id="Txt_2"]').send_keys('sclossolar')
C:\Users\owner\AppData\Local\Temp/ipykernel_16924/1003527085.py:30: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  self.driver.find_element_by_xpath('//*[@id="imageField"]').click()
C:\